In [ ]:
#pip install tsfresh
!pip install gcsfs openpyxl
!pip install hdbscan


In [ ]:
import pandas as pd
import numpy as np
from sklearn.utils import resample
#from tsfresh import extract_features
#from tsfresh.utilities.dataframe_functions import impute

In [ ]:
"""
reducing.py
Author: Kirgsn, 2018
https://www.kaggle.com/code/etremblay/fail-safe-parallel-memory-reduction/notebook?scriptVersionId=27504964
https://wkirgsn.github.io/2018/02/10/auto-downsizing-dtypes/
[alternative] https://www.kaggle.com/gemartin/load-data-reduce-memory-usage
              (tener en cuenta) https://www.kaggle.com/c/champs-scalar-coupling/discussion/96655

Use like this:
>>> from reducing import Reducer
>>> df = Reducer().reduce(df)
"""
import numpy as np
import pandas as pd
import time
import gc
# from joblib import Parallel, delayed
# from tqdm import tqdm

__all__ = ["Reducer"]


class Reducer:
    """
    Class that takes a dict of increasingly big numpy datatypes to transform
    the data of a pandas dataframe into, in order to save memory usage.
    """

    memory_scale_factor = 1024**2  # memory in MB

    def __init__(self, conv_table=None, use_categoricals=True, n_jobs=-1):
        """
        :param conv_table: dict with np.dtypes-strings as keys
        :param use_categoricals: Whether the new pandas dtype "Categoricals"
                shall be used
        :param n_jobs: Parallelization rate
        """

        self.conversion_table = conv_table or {
            "int": [np.int8, np.int16, np.int32, np.int64],
            "uint": [np.uint8, np.uint16, np.uint32, np.uint64],
            "float": [np.float32],
            "datetime": []
        }
        self.null_int = [
            pd.Int8Dtype,
            pd.Int16Dtype,
            pd.Int32Dtype,
            pd.Int64Dtype,
            pd.UInt8Dtype,
            pd.UInt16Dtype,
            pd.UInt32Dtype,
            pd.UInt64Dtype,
        ]

        self.use_categoricals = use_categoricals
        self.n_jobs = n_jobs

    def _type_candidates(self, k):
        for c in self.conversion_table[k]:
            i = np.iinfo(c) if "int" in k else np.finfo(c)
            yield c, i


    def reduce(self, df, verbose=False):
        """Takes a dataframe and returns it with all data transformed to the
        smallest necessary types.

        :param df: pandas dataframe
        :param verbose: If True, outputs more information
        :return: pandas dataframe with reduced data types
        """
        mem_usage_orig = df.memory_usage().sum() / self.memory_scale_factor
        start_time = time.time()

        gc.enable()
        df_reduced = df.apply(self._reduce, axis=0, args=(verbose,), n_jobs=self.n_jobs)

        del df
        gc.collect()

        mem_usage_new = df_reduced.memory_usage().sum() / self.memory_scale_factor
        end_time = time.time()
        print(
            f"Reduced file from {mem_usage_orig:.2f} MB "
            f"to {mem_usage_new:.2f} MB "
            f"in {(end_time - start_time):.2f} seconds"
        )

        return df_reduced

    def _reduce(self, s, verbose, n_jobs):
        try:
            isnull = False
            # skip NaNs
            if s.isnull().any():
                isnull = True
            # detect kind of type
            coltype = s.dtype
            if np.issubdtype(coltype, np.integer):
                conv_key = "int" if s.min() < 0 else "uint"
            elif np.issubdtype(coltype, np.floating):
                conv_key = "float"
            else:
                conv_key = coltype.name

            # get smallest type
            if conv_key in self.conversion_table:
                candidates = self._type_candidates(conv_key)
                for i, (t, info) in enumerate(candidates):
                    if s.max() <= info.max and s.min() >= info.min:
                        if verbose:
                            print(f"{s.name}: {coltype} -> {t}")
                        if isnull:
                            s = s.astype(t, copy=False)
                            s = s.where(s.notnull(), None)
                        else:
                            s = s.astype(t, copy=False)
                        break
            elif self.use_categoricals and "category" not in conv_key:
                if verbose:
                    print(f"{s.name}: {coltype} -> category")
                s = s.astype("category")
            return s
        except Exception:
            print(f"Error in column {s.name}: {sys.exc_info()[0]}")
            return s

In [ ]:
file_path = 'gs://marcevera/datasets/competencia_02.csv.gz'
df = pd.read_csv(file_path)
df = Reducer().reduce(df)

In [ ]:
undersampled_data = pd.DataFrame()
for month in df.foto_mes.unique():
    
    # Separar los bloques por clases
    clase_baja_1 = df[(df['foto_mes'] == month) & (df['clase_ternaria'] == 'BAJA+1')]
    clase_baja_2 = df[(df['foto_mes'] == month) & (df['clase_ternaria'] == 'BAJA+2')]
    clase_continua = df[(df['foto_mes'] == month) & (df['clase_ternaria'] == 'CONTINUA')]

    undersampled_continua = resample(clase_continua, replace=False, n_samples=max(10, int((len(clase_baja_1)+len(clase_baja_2))/2)), random_state=42)

    # Concatenar los bloques undersampleados al DataFrame final
    undersampled_data = pd.concat([undersampled_data, clase_baja_1, clase_baja_2, undersampled_continua])

In [ ]:
# Definir el tamaño del bloque para procesar en cada iteración
block_size = 10000

# Inicializar un DataFrame vacío para almacenar los resultados undersampleados
undersampled_data = pd.DataFrame()

# Leer el archivo CSV en bloques
for chunk in pd.read_csv(file_path, chunksize=block_size):

    # Separar los bloques por clases
    clase_baja_1 = chunk[chunk['clase_ternaria'] == 'BAJA+1']
    clase_baja_2 = chunk[chunk['clase_ternaria'] == 'BAJA+2']
    clase_continua = chunk[chunk['clase_ternaria'] == 'CONTINUA']

    undersampled_continua = resample(clase_continua, replace=False, n_samples=max(10, int((len(clase_baja_1)+len(clase_baja_2))/2)), random_state=42)

    # Concatenar los bloques undersampleados al DataFrame final
    undersampled_data = pd.concat([undersampled_data, clase_baja_1, clase_baja_2, undersampled_continua])


In [ ]:
df = undersampled_data.copy()

In [ ]:
diccionario_datos  = pd.read_excel('gs://marcevera/datasets/DiccionarioDatos_2023.xlsx')

In [ ]:
columnas_pesos = diccionario_datos[diccionario_datos['unidad'] == 'pesos']['campo'].tolist()

In [ ]:
# Crear nuevas columnas divididas por el promedio para cada foto_mes
for columna in df.columns:
    # columna = columna.lower()
    if columna in columnas_pesos:
        promedio_por_mes = df.groupby('foto_mes')[columna].transform('mean')
        nueva_columna = f'{columna}_div_promedio'
        df[nueva_columna] = df[columna] / promedio_por_mes
        df = df.drop(columna, axis=1)

In [ ]:
# 1. Convertir la columna 'foto_mes' a formato de fecha
df['foto_mes'] = pd.to_datetime(df['foto_mes'], format='%Y%m')

# 2. Asegurarte de tener una columna 'id'
#df['id'] = df['numero_de_cliente']

# 3. Identificar columnas numéricas para aplicar interpolación
columnas_numericas = df.select_dtypes(include='number').columns

# 4. Aplicar interpolación solo a columnas numéricas
df[columnas_numericas] = df[columnas_numericas].interpolate(method='linear', axis=0)
df = df.ffill().bfill()

In [ ]:
df_numeric = df[columnas_numericas]
df_numeric['foto_mes'] = df['foto_mes']

# 6. Extraer características temporales con TSFRESH
extracted_features = extract_features(df_numeric, column_id='id', column_sort='foto_mes', n_jobs=16)

# 7. Agregar la columna 'clase_ternaria' a las características extraídas
extracted_features['clase_ternaria'] = df.groupby('id')['clase_ternaria'].first()

# 8. Imputar valores faltantes
extracted_features = impute(extracted_features)

# Ahora, `extracted_features` contiene las características temporales generadas por TSFRESH con imputación de valores NaN

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import classification_report, accuracy_score

# Convertir la columna 'clase_ternaria' a valores numéricos (por ejemplo, 0 para CONTINUA, 1 para BAJA+1, 2 para BAJA+2)
df['clase_ternaria'] = df['clase_ternaria'].map({'CONTINUA': 0, 'BAJA+1': 1, 'BAJA+2': 2})

# Dividir el dataset en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(df.drop(['clase_ternaria', 'foto_mes'], axis=1), df['clase_ternaria'], test_size=0.2, random_state=42)

# Entrenar un modelo de clasificación (por ejemplo, Random Forest)
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Evaluar el modelo en el conjunto de prueba
y_pred = clf.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

# Visualizar la importancia de las características
feature_importances = clf.feature_importances_
feature_names = X_train.columns
feature_importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': feature_importances})
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

# Visualizar las 15 características más importantes
plt.figure(figsize=(12, 8))
sns.barplot(x='Importance', y='Feature', data=feature_importance_df.head(15), palette='viridis')
plt.title('Top 15 Características Importantes')
plt.show()

# Seleccionar características importantes
sfm = SelectFromModel(clf, threshold=0.01)
sfm.fit(X_train, y_train)
selected_features = X_train.columns[sfm.get_support()]

# Imprimir las características seleccionadas
print("\nCaracterísticas seleccionadas:")
print(selected_features)

In [ ]:
df_filtered.shape

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import hdbscan
from sklearn.decomposition import PCA
from sklearn.feature_selection import VarianceThreshold

# Cargar el dataset
#file_path = 'ruta/del/tu/dataset.csv'  # Reemplaza con la ruta real de tu archivo CSV
#df = pd.read_csv(file_path)

# Reemplazar NaN con valores adecuados (puedes usar la media o la mediana)
#df.fillna(df.mean(), inplace=True)

df_filtered = df[df['clase_ternaria'].isin(['BAJA+2', 'CONTINUA'])]

var_thresh = VarianceThreshold(threshold=0.1)
transformed_df = var_thresh.fit_transform(df_filtered.select_dtypes('number'))
# Filtrar filas relevantes solo para 'BAJA+2' y 'CONTINUA'
#df_filtered = transformed_df[transformed_df['clase_ternaria'].isin(['BAJA+2', 'CONTINUA'])]

# Seleccionar características relevantes (puedes personalizar esta parte según tus necesidades)
features = transformed_df
#.drop(['clase_ternaria', 'foto_mes'], axis=1)

# Estandarizar las características
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

# Reducción de dimensionalidad con PCA (puedes ajustar el número de componentes según tus necesidades)
pca = PCA(n_components=2)
features_pca = pca.fit_transform(features_scaled)

# Aplicar HDBSCAN
clusterer = hdbscan.HDBSCAN(min_cluster_size=50, min_samples=5)
clusters = clusterer.fit_predict(features_pca)

# Visualizar los resultados
plt.figure(figsize=(10, 8))
sns.scatterplot(x=features_pca[:, 0], y=features_pca[:, 1], hue=clusters, palette='viridis', legend='full')
plt.title('HDBSCAN Clustering Result')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.legend(title='Cluster')
plt.show()


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import hdbscan
from sklearn.decomposition import PCA

# Cargar el dataset
#file_path = 'ruta/del/tu/dataset.csv'  # Reemplaza con la ruta real de tu archivo CSV
#df = pd.read_csv(file_path)

# Reemplazar NaN con valores adecuados (puedes usar la media o la mediana)
#df.fillna(df.mean(), inplace=True)

# Convertir la columna 'clase_ternaria' a valores numéricos (por ejemplo, 0 para CONTINUA, 1 para BAJA+1, 2 para BAJA+2)
df['clase_ternaria'] = df['clase_ternaria'].map({'CONTINUA': 0, 'BAJA+1': 1, 'BAJA+2': 2})

# Estandarizar las características
scaler = StandardScaler()
features_scaled = scaler.fit_transform(df.drop(['foto_mes', 'clase_ternaria'], axis=1))

# Reducción de dimensionalidad con PCA (puedes ajustar el número de componentes según tus necesidades)
pca = PCA(n_components=2)
features_pca = pca.fit_transform(features_scaled)

# Crear un DataFrame para almacenar los resultados de cada mes
results_by_month = pd.DataFrame(columns=['foto_mes', 'cluster'])

# Iterar sobre los meses
for month in df['foto_mes'].unique():
    # Filtrar el DataFrame por el mes actual
    df_month = df[df['foto_mes'] == month]

    # Seleccionar características relevantes (puedes personalizar esta parte según tus necesidades)
    features_month = df_month.drop(['foto_mes', 'clase_ternaria'], axis=1)

    # Estandarizar las características
    features_scaled_month = scaler.transform(features_month)

    # Aplicar HDBSCAN
    clusterer = hdbscan.HDBSCAN(min_cluster_size=50, min_samples=5)
    clusters = clusterer.fit_predict(features_scaled_month)

    # Almacenar los resultados en el DataFrame
    results_by_month = pd.concat([results_by_month, pd.DataFrame({'foto_mes': [month] * len(clusters), 'cluster': clusters})])

# Visualizar los resultados globales
plt.figure(figsize=(12, 6))
sns.scatterplot(x=features_pca[:, 0], y=features_pca[:, 1], hue=results_by_month['cluster'], palette='viridis', legend='full')
plt.title('HDBSCAN Clustering Result - Global')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.legend(title='Cluster')
plt.show()

# Visualizar los resultados por mes
plt.figure(figsize=(16, 8))
sns.scatterplot(x=features_pca[:, 0], y=features_pca[:, 1], hue=results_by_month['foto_mes'], palette='viridis', legend='full')
plt.title('HDBSCAN Clustering Result - Por Mes')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.legend(title='Mes')
plt.show()


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import hdbscan
import umap
import os

# Cargar el dataset
#file_path = 'ruta/del/tu/dataset.csv'  # Reemplaza con la ruta real de tu archivo CSV
#df = pd.read_csv(file_path)

# Reemplazar NaN con valores adecuados (puedes usar la media o la mediana)
#df.fillna(df.mean(), inplace=True)

# Convertir la columna 'clase_ternaria' a valores numéricos (por ejemplo, 0 para CONTINUA, 1 para BAJA+1, 2 para BAJA+2)
#df['clase_ternaria'] = df['clase_ternaria'].map({'CONTINUA': 0, 'BAJA+1': 1, 'BAJA+2': 2})

# Crear un directorio para almacenar los gráficos
output_directory = 'graficos_por_mes'
os.makedirs(output_directory, exist_ok=True)

# Iterar sobre los meses
for month in df['foto_mes'].unique():
    # Filtrar el DataFrame por el mes actual
    df_month = df[df['foto_mes'] == month]

    # Seleccionar características relevantes (puedes personalizar esta parte según tus necesidades)
    features_month = df_month.drop(['foto_mes', 'clase_ternaria'], axis=1)

    # Estandarizar las características
    scaler = StandardScaler()
    features_scaled_month = scaler.fit_transform(features_month)

    # Aplicar UMAP para reducción de dimensionalidad
    reducer = umap.UMAP(n_components=2)
    features_umap = reducer.fit_transform(features_scaled_month)

    # Aplicar HDBSCAN
    clusterer = hdbscan.HDBSCAN(min_cluster_size=50, min_samples=5)
    clusters = clusterer.fit_predict(features_scaled_month)

    # Almacenar el resultado del clustering en el DataFrame original
    df_month['cluster'] = clusters

    # Visualizar los resultados por mes
    plt.figure(figsize=(12, 6))
    sns.scatterplot(x=features_umap[:, 0], y=features_umap[:, 1], hue=clusters, palette='viridis', legend='full')
    plt.title(f'HDBSCAN Clustering Result - Mes {month}')
    plt.xlabel('UMAP Component 1')
    plt.ylabel('UMAP Component 2')
    plt.legend(title='Cluster')
    
    # Guardar el gráfico en el directorio
    plt.savefig(os.path.join(output_directory, f'clustering_result_mes_{month}.png'))
    plt.close()